## Lab 2 – Datarensning (ETL) för Smart Home IoT – “dirty version”
# Mål med labben

I denna labb ska du:

* Läsa in ett smutsigt IoT-dataset för

* smarta hem.

* Undersöka datakvalitet:

* saknade värden

* dubbletter

* orimliga värden (outliers)

* Rensa och standardisera datan    (Transform i ETL).

Slutligen Spara en ren version av datan för Lab 3 & 4 (Load).

Steg 0 – Importera bibliotek

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Steg 1 – Läs in smutsig data

För att läsa in och spara datafiler, finns det två sätt via Git eller Google driver. Förlj länken nedan där har jag förklarat hur man gör beroende på vilken väg ni väljer
https://www.notion.so/Lab2-Datarensning-2aa9de1f0d8380ab9e64e60eb26c7656?source=copy_link

In [ ]:
# läsa in den smutsiga datan, sparade från lab 1

In [9]:
print(f"  • {200} saknade värden (NaN)")
print(f"  • {150} dubbletter")
print(f"  • {100} extrema värden (outliers)")
print(f"\n Nu ska vi lära oss hitta och fixa dessa problem!")

  • 200 saknade värden (NaN)
  • 150 dubbletter
  • 100 extrema värden (outliers)

 Nu ska vi lära oss hitta och fixa dessa problem!


In [12]:
# Visa de första raderna
df = pd.read_csv("../data/Test.csv")
df.head()

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description
0,2017-05-18 00:00:00,NaN,73,63,1,27,4,4,285.15,0.0,0,90,Rain,moderate rain
1,2017-05-18 00:00:00,NaN,251,63,1,27,4,4,285.15,0.0,0,90,Mist,mist
2,2017-05-18 00:00:00,NaN,75,56,1,0,1,1,285.15,0.0,0,90,Drizzle,light intensity drizzle
3,2017-05-18 01:00:00,NaN,98,56,1,351,2,2,284.79,0.0,0,90,Rain,heavy intensity rain
4,2017-05-18 01:00:00,NaN,283,56,1,351,1,1,284.79,0.0,0,90,Mist,mist


Steg 2 – Överblick av datasetet

In [24]:
df.info()

df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14454 entries, 0 to 14453
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date_time            14454 non-null  object 
 1   is_holiday           18 non-null     object 
 2   air_pollution_index  14454 non-null  int64  
 3   humidity             14454 non-null  int64  
 4   wind_speed           14454 non-null  int64  
 5   wind_direction       14454 non-null  int64  
 6   visibility_in_miles  14454 non-null  int64  
 7   dew_point            14454 non-null  int64  
 8   temperature          14454 non-null  float64
 9   rain_p_h             14454 non-null  float64
 10  snow_p_h             14454 non-null  int64  
 11  clouds_all           14454 non-null  int64  
 12  weather_type         14454 non-null  object 
 13  weather_description  14454 non-null  object 
dtypes: float64(2), int64(8), object(4)
memory usage: 1.5+ MB


,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all
count,14454.000000,14454.000000,14454.000000,14454.000000,14454.000000,14454.000000,14454.000000,14454.000000,14454.0,14454.000000
mean,154.679397,67.833887,3.607306,209.285319,5.005535,5.005535,283.859084,0.066965,0.0,46.801785
std,83.405392,21.147425,2.212177,94.996443,2.586276,2.586276,12.769255,0.466853,0.0,39.232165
min,10.000000,10.000000,0.000000,0.000000,1.000000,1.000000,246.150000,0.000000,0.0,0.000000
25%,83.000000,54.000000,2.000000,140.000000,3.000000,3.000000,273.720000,0.000000,0.0,1.000000
50%,155.000000,71.000000,3.000000,211.000000,5.000000,5.000000,287.725000,0.000000,0.0,40.000000
75%,227.000000,85.000000,5.000000,290.000000,7.000000,7.000000,293.860000,0.000000,0.0,90.000000
max,299.000000,100.000000,16.000000,360.000000,9.000000,9.000000,310.070000,10.600000,0.0,92.000000


Steg 3 – Kontrollera datakvalitet

4.1 Saknade värden

In [26]:
print("Saknade värden per kolumn")
display(df.isna().sum())  # isna().sum() räknar hur många NaN finns det i varje kolumn

Saknade värden per kolumn


date_time                  0
is_holiday             14436
air_pollution_index        0
humidity                   0
wind_speed                 0
wind_direction             0
visibility_in_miles        0
dew_point                  0
temperature                0
rain_p_h                   0
snow_p_h                   0
clouds_all                 0
weather_type               0
weather_description        0
dtype: int64

4.2 Dubbletter

In [25]:
print("Antal dubbletter (hela rader som är kopior):", 
df.duplicated().sum())
dups = df[df.duplicated()]


Antal dubbletter (hela rader som är kopior): 0


In [ ]:
dups['Home ID'].value_counts() # om man inte vilka ex home id som är duplicated

In [ ]:
dups[dups['Home ID'] == 94]


4.3 Orimliga värden (business rules)

In [30]:
# kollar först outliers för energyforbrukning
energy_outliers = df[(df['air_pollution_index'] < 0) |
                     (df['air_pollution_index'] > 1000)]
print("Antal orimliga energivärden:", len(energy_outliers))
#energy_ouliners

Antal orimliga energivärden: 0


In [ ]:
# kollar outliers för temp
temp_outliers = df[(df['Temp_Kök_C'] < -50) |
                   (df['Temp_Kök_C'] > 60)]
print("Antal orimliga temperaturer:", len(temp_outliers))
temp_outliners

Steg 5 – Visualisera problemet (outlier-spik)

In [ ]:
# Graf 1: Visa BARA de extrema temperaturerna (över 50°C)
extrema_temp = df[data['Temp_Kök_C'] > 50]['Temp_Kök_C']
extrema_temp.hist(color='red')
plt.title('PROBLEM: Extrema temperaturer!')
plt.xlabel('Temperatur (°C)')
plt.ylabel('Antal')
plt.show()

In [ ]:
print(" GRAF 1 VISAR:")
print(f"• {len(extrema_temp)} temperaturer vid {extrema_temp.iloc[0]:.0f}°C")
print("• Det är för varmt för ett kök! Huset skulle brinna!")

In [ ]:

# Graf 2: Visa BARA de negativa energivärdena  
negativa_energi = data[data['Vitvaror_Energi_Wh'] < 0]['Vitvaror_Energi_Wh']
negativa_energi.hist(color='orange')
plt.title('PROBLEM: Negativa energivärden!')
plt.xlabel('Energi (Wh)')  
plt.ylabel('Antal')
plt.show()

In [ ]:
print(" GRAF 2 VISAR:")
print(f"• {len(negativa_energi)} negativa energivärden vid {negativa_energi.iloc[0]:.0f} Wh")
print("• Vitvaror kan inte ge tillbaka energi - det är omöjligt!")

Steg 6 – Rensa datan

In [5]:
# Låt oss först kolla vilka extrema värden vi har
print("🔍 EXTREMA VÄRDEN I DATAN:")
print(f"Högsta temperatur i kök: {data['Temp_Kök_C'].max():.1f}°C")
print(f"Lägsta temperatur i kök: {data['Temp_Kök_C'].min():.1f}°C")
print(f"Högsta energi: {df['Vitvaror_Energi_Wh'].max():.1f} Wh")
print(f"Lägsta energi: {df['Vitvaror_Energi_Wh'].min():.1f} Wh")

# Kolla hur många extrema värden vi har
temp_extrema = (data['Temp_Kök_C'] > 50).sum()
energi_negativ = (data['Vitvaror_Energi_Wh'] < 0).sum()
print(f"\nAntal temperaturer över 50°C: {temp_extrema}")
print(f"Antal negativa energivärden: {energi_negativ}")

🔍 EXTREMA VÄRDEN I DATAN:


NameError: name 'data' is not defined

6.1 – Hantera saknade värden, och NaN


6.1.1 Energy Consumption (kWh)

In [ ]:
# skapar en kopia av datan
df_cleaned = df_home.copy()

In [ ]:
print(f"Före rensning: {len(df_cleaned)} rader")

# 1. Beräkna medelvärden (skippar automatiskt NaN värden)
energy_mean = df_cleaned['Vitvaror_Energi_Wh'].mean()

# 2. Fylla ALLA saknade värden (NaN, tomma celler) med medelvärden
missing_energy_before = df_cleaned['Vitvaror_Energi_Wh'].isnull().sum()
df_cleaned['Vitvaror_Energi_Wh'].fillna(energy_mean, inplace=True)

print(f"Fyllde {missing_energy_before} energi-NaN med medelvärde: {energy_mean:.2f} kWh")

6.1.2 Outdoor Temperature (°C)

In [ ]:
print(f"Före rensning: {len(df_cleaned)} rader")

# 1. Beräkna medelvärden (skippar automatiskt NaN värden)
temp_mean = df_cleaned['Temp_Kök_C'].mean()

# 2. Fylla ALLA saknade värden (NaN, tomma celler) med medelvärden
missing_temp_before = df_cleaned['Temp_Kök_C'].isnull().sum()

df_cleaned['Temp_Kök_C'].fillna(temp_mean, inplace=True)

print(f"Fyllde {missing_temp_before} temperatur-NaN med medelvärde: {temp_mean:.1f}°C")

6.3 Hantera outliers

6.2.1 Outlier Energy Consumption

In [ ]:
#  Ersätt outliers och felaktiga värden med medelvärden

# Ersätt negativ energi och extremvärden (>15 kWh) med medelvärde
energy_outliers = (df_home_cleaned['Energy Consumption (kWh)'] < 0) | (df_home_cleaned['Energy Consumption (kWh)'] > 15)
outlier_count_energy = energy_outliers.sum()
df_home_cleaned.loc[energy_outliers, 'Energy Consumption (kWh)'] = energy_mean
print(f"Ersatte {outlier_count_energy} energi-outliers med medelvärde")

6.2.2 Oulier Outdoor Temperature

In [ ]:
# Ersätt extrema temperaturer (<-40 eller >50) med medelvärde
temp_outliers = (df_home_cleaned['Outdoor Temperature (°C)'] < -40) | (df_home_cleaned['Outdoor Temperature (°C)'] > 50)
outlier_count_temp = temp_outliers.sum()
df_home_cleaned.loc[temp_outliers, 'Outdoor Temperature (°C)'] = temp_mean
print(f"Ersatte {outlier_count_temp} temperatur-outliers med medelvärde")

print(f"Alla rader behållna: {len(df_home_cleaned)} rader")

6.3 Ta bort alla dubbletter

In [4]:
# 2. Ta bort dubbletter
df_home_cleaned = df_home_cleaned.drop_duplicates()
print(f"Efter dubbletter: {len(df_home_cleaned)} rader")

NameError: name 'df_home_cleaned' is not defined

Steg 7 – Hitta felaktiga textvärden innan rensning

7.1 Hitta unika värden i textkolumnerna

In [ ]:
print("Unika värden i Appliance Type:")
display(df_clean['Appliance Type'].unique())

print("\nUnika värden i Season:")
display(df_clean['Season'].unique())

7.2 Hitta värden som innehåller mellanslag före/efter

In [ ]:
mask_spaces = df_clean['Appliance Type'].str.contains(r'^\s|\s$', regex=True)
df_clean[mask_spaces][['Appliance Type', 'Home ID', 'Date', 'Time']]

7.3 Hitta värden med blandning av stora och små bokstäver

In [ ]:
mask_case = df_clean['Appliance Type'].str.match(r'[A-Za-z ]+') & ~df['Appliance Type'].str.istitle()
df_clean[mask_case][['Appliance Type', 'Home ID']]


Steg 8– Standardisera text och datatyper

In [ ]:
# standardisera texfält- ta bort mellanslag, rättar case
df['Appliance Type'] = df['Appliance Type'].str.strip().str.title()
df['Season'] = df['Season'].str.strip().str.capitalize()


In [ ]:
# säkerställer att datum är date time format
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M').dt.time

In [ ]:
#vi kan validera att datan har rensat
dups = df[df.duplicated(keep=False)]
dups['Home ID'].value_counts()

Steg 9 - Validera rensad data

In [ ]:
# Är rensningen OK?
print(f"\n KLART! Slutlig ren data: {len(data_fixed)} rader")
print(f" Minskade från {len(df_home)} till {len(data_fixed)} rader")


print(f" Saknade värden: {df_home.isnull().sum().sum()}")
print(f" Dubbletter: {df_home.duplicated().sum()}")
print(f" Negativ energi: {(df_home['Vitvaror_Energi_Wh'] < 0).sum()}")
print("Nu har vi perfekt data att analysera! ")

df_cleaned.head()

Steg 10 – Spara ren version

In [ ]:

# spara rensat data till csv file
df_cleaned.to_csv(
    '/content/drive/MyDrive/big_data_labs/cleaned_iot_data.csv',
    index=False
)

print("Ren version sparad som cleaned_iot_data.csv")

Steg 11 –  Beräkna Rolling Mean & Diff på ren data

In [ ]:
# Sortera i tidsordning
df = df.sort_values('DateTime')

# Rullande medelvärde
df['RollingMean'] = df['Energy Consumption (kWh)'].rolling(window=50).mean()

# Differens mellan mätpunkter
df['Diff'] = df['Energy Consumption (kWh)'].diff()

In [ ]:
# visa grafen efter rensning
plt.figure(figsize=(12,6))

plt.plot(df['DateTime'].head(500),
         df['Energy Consumption (kWh)'].head(500),
         label='Rådata', alpha=0.3)

plt.plot(df['DateTime'].head(500),
         df['RollingMean'].head(500),
         label='Rullande medelvärde', linewidth=2)

plt.xlabel('Tid')
plt.ylabel('kWh')
plt.title('Energiförbrukning – efter datarensning')
plt.legend()

# Fix för x-axeln:
ax = plt.gca()
ax.xaxis.set_major_locator(plt.MaxNLocator(10))  # Max 10 etiketter
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
